In [ ]:
import numpy as np
import matplotlib.pyplot as plt

from scipy import ndimage

%matplotlib inline

In [ ]:
from skimage import io  # skimage's I/O submodule.
from skimage import data

In [ ]:
img = io.imread('bead_pack.tif')

In [ ]:
!wget https://github.com/dani-lbnl/imagexd19/blob/master/dip/data/bead_pack.tif?raw=true

In [ ]:
!mv bead_pack.tif?raw=true bead_pack.tif

In [ ]:
!ls *.tif

In [ ]:
img = io.imread('bead_pack.tif')

## Basic image summary

In [ ]:
print('* Shape: {}'.format(img.shape))
print('* Type: {}'.format(img.dtype))
print('* Range: {}, {}'.format(img.min(), img.max()))

## Skim through

In [ ]:
import ipywidgets as widgets
from IPython.display import display
from ipywidgets import interact, interactive, fixed, interact_manual

def slicer(z):
    plt.imshow(img[z,:,:], cmap='gray')

interact(slicer, z=widgets.IntSlider(min=0,max=60,step=1,value=5));

## [skimage.exposure](https://scikit-image.org/docs/stable/api/skimage.exposure.html) - evaluating or changing the exposure of an image<a id='exposure'></a>

This module contains a number of functions for adjusting image contrast. We will use `exposure.adjust_gamma`, which performs gamma correction in the input image.


[Gamma correction](https://en.wikipedia.org/wiki/Gamma_correction), also known as Power Law Transform, brightens or darkens an image. The function $O = I^\gamma$ is applied to each pixel in the image. A `gamma < 1` will brighten an image, while a `gamma > 1` will darken an image.

In [ ]:
from skimage import exposure
ex = exposure.equalize_hist(img)
def slicer(z):
    plt.imshow(ex[z,:,:], cmap='gray')

interact(slicer, z=widgets.IntSlider(min=0,max=60,step=1,value=5));

## Edge detection

[Edge detection](https://en.wikipedia.org/wiki/Edge_detection) highlights regions in the image where a sharp change in contrast occurs. The intensity of an edge corresponds to the steepness of the transition from one intensity to another. A gradual shift from bright to dark intensity results in a dim edge. An abrupt shift results in a bright edge.

## [skimage.filters](https://scikit-image.org/docs/stable/api/skimage.filters.html) - apply filters to an image<a id='filters'></a>

Filtering applies whole-image modifications such as sharpening or blurring. In addition to edge detection, `skimage.filters` provides functions for filtering and thresholding images.

Notable functions include (links to relevant gallery examples):

* [Thresholding](https://scikit-image.org/docs/stable/auto_examples/applications/plot_thresholding.html):
  * `filters.threshold_*` (multiple different functions with this prefix)
  * `filters.try_all_threshold` to compare various methods
* [Edge finding/enhancement](https://scikit-image.org/docs/stable/auto_examples/edges/plot_edge_filter.html):
  * `filters.sobel` - not adapted for 3D images. It can be applied planewise to approximate a 3D result.
  * `filters.prewitt`
  * `filters.scharr`
  * `filters.roberts`
  * `filters.laplace`
  * `filters.hessian`
* [Ridge filters](https://scikit-image.org/docs/stable/auto_examples/edges/plot_ridge_filter.html):
  * `filters.meijering`
  * `filters.sato`
  * `filters.frangi`
* Inverse filtering (see also [skimage.restoration](#restoration)):
  * `filters.weiner`
  * `filters.inverse`
* [Directional](https://scikit-image.org/docs/stable/auto_examples/features_detection/plot_gabor.html): `filters.gabor`
* Blurring/denoising
  * `filters.gaussian`
  * `filters.median`
* [Sharpening](https://scikit-image.org/docs/stable/auto_examples/filters/plot_unsharp_mask.html): `filters.unsharp_mask`
* Define your own filter: `LPIFilter2D`
  
The sub-submodule `skimage.filters.rank` contains rank filters. These filters are nonlinear and operate on the local histogram.

In [ ]:
from skimage import filters
img2 = filters.laplace(img)
def slicer(z):
    plt.imshow(img2[z,:,:], cmap='gray')

interact(slicer, z=widgets.IntSlider(min=0,max=60,step=1,value=5));

In [ ]:
from skimage import filters
n=27
aslice = img[n,:,:]
filters.try_all_threshold(aslice,figsize=(16,10))

In [ ]:
from skimage import filters
img3 = filters.gaussian(img)
t=filters.threshold_isodata(img3)
img3 = img3>t

def slicer(z):
    plt.imshow(img3[z,:,:], cmap='gray')

interact(slicer, z=widgets.IntSlider(min=0,max=60,step=1,value=5));

## <a id='morphology'></a>[skimage.morphology](https://scikit-image.org/docs/stable/api/skimage.morphology.html) - binary and grayscale morphology

Morphological image processing is a collection of non-linear operations related to the shape or morphology of features in an image, such as boundaries, skeletons, etc. In any given technique, we probe an image with a small shape or template called a structuring element, which defines the region of interest or neighborhood around a pixel.

[Mathematical morphology](https://en.wikipedia.org/wiki/Mathematical_morphology) operations and structuring elements are defined in `skimage.morphology`. Structuring elements are shapes which define areas over which an operation is applied. The response to the filter indicates how well the neighborhood corresponds to the structuring element's shape.

There are a number of two and three dimensional structuring elements defined in `skimage.morphology`. Not all 2D structuring element have a 3D counterpart. The simplest and most commonly used structuring elements are the `disk`/`ball` and `square`/`cube`.

In [ ]:
from skimage import morphology  # skimage's morphological submodules.

In [ ]:
ball = morphology.ball(radius=3)

In [ ]:
img2te=morphology.binary_erosion(img3,selem=ball)
img2to=morphology.binary_opening(img3,selem=ball)

In [ ]:
#show two 2D images side by side for quick comparison
def imshowcmp(before,after,lut):
    f, ax = plt.subplots(1, 2, figsize=(20, 20))
    ax[0].imshow(before,cmap=lut)
    ax[1].imshow(after,cmap=lut)

In [ ]:
imshowcmp(img2te[n,:,:],img2to[n,:,:],'gray')

It's clear that we need to improve the segmentation before assigning different labels to disjoint regions as believed in the original sample.

Watershed segmentation can distinguish touching objects. Markers are placed at local minima and expanded outward until there is a collision with markers from another region. The inverse intensity image transforms bright cell regions into basins which should be filled.

In declumping, markers are generated from the distance function. Points furthest from an edge have the highest intensity and should be identified as markers using skimage.feature.peak_local_max. Regions with pinch points should be assigned multiple markers.

In [ ]:
binary=morphology.binary_erosion(img2to,selem=ball)



In [ ]:
a = binary
def slicer(z):
    plt.imshow(a[z,:,:], cmap='gray')
interact(slicer, z=widgets.IntSlider(min=0,max=60,step=1,value=5));

## Unsupervised classification - flood fill

In [ ]:
from skimage import segmentation as seg
b = filters.median(img[n,:,:])
plt.imshow(b)

In [ ]:
seed_point = (10, 10)  # Experiment with the seed point
c = seg.flood(b, seed_point, tolerance=50)
plt.imshow(c)

## Unsupervised - kmeans

In [ ]:
from sklearn import cluster
kmeans_cluster = cluster.KMeans(n_clusters=50)
kmeans_cluster.fit(b)
cluster_centers = kmeans_cluster.cluster_centers_
cluster_labels = kmeans_cluster.labels_

In [ ]:
plt.imshow(cluster_centers[cluster_labels])

In [ ]:
import itk
from itkwidgets import view

In [ ]:
image_itk = itk.GetImageFromArray(binary.astype(np.uint8))
view(image_itk, cmap='Cold and Hot', slicing_planes=True,gradient_opacity=0.4)

In [ ]:
from skimage import feature, measure

In [ ]:
label = measure.label(binary)
regions = measure.regionprops(label,intensity_image=img)

In [ ]:
np.max(label) #len(regionprops)

In [ ]:
#all_props = {p:regions[0][p] for p in regions[0] if p not in ('image','convex_image','filled_image')}
for p in regions:
    print(p.area)

In [ ]:
image_itk = itk.GetImageFromArray(label.astype(np.uint16))
view(image_itk, cmap='Cold and Hot', slicing_planes=True,gradient_opacity=0.4)

## How about some deep learning?

- Inception v3 is a widely-used image recognition model that has been shown to attain greater than 78.1% accuracy on the ImageNet dataset. The model is the culmination of many ideas developed by multiple researchers over the years.
- Inception-v3 is a convolutional neural network that is trained on more than a million images from the ImageNet database. The network is **48** layers deep and can classify images into **1000 object** categories, such as keyboard, mouse, pencil, and many animals.
- There are a total of 1,281,167 images for training. The number of images for each synset (category) ranges from 732 to 1300. There are 50,000 validation images, with 50 images per synset. There are 100,000 test images.

In [ ]:
from tensorflow.keras.applications.inception_v3 import InceptionV3, decode_predictions

In [ ]:
net = InceptionV3()

**Why is it so fast?** We used transfer learning, a machine learning method which utilizes a pre-trained neural network. For example, the image recognition model called Inception-v3 consists of two parts: 
- Feature extraction part with a convolutional neural network;
- Classification part with fully-connected and softmax layers.

In [ ]:
from skimage import transform

def inception_predict(image):
    # Rescale image to 299x299, as required by InceptionV3
    image_prep = transform.resize(image, (299, 299, 3), mode='reflect')
    
    # Scale image values to [-1, 1], as required by InceptionV3
    image_prep = (img_as_float(image_prep) - 0.5) * 2
    
    predictions = decode_predictions(
        net.predict(image_prep[None, ...])
    )
    
    plt.imshow(image, cmap='gray')
    
    for pred in predictions[0]:
        (n, klass, prob) = pred
        print(f'{klass:>15} ({prob:.3f})')



In [ ]:
from skimage import data, img_as_float
inception_predict(data.chelsea())

In [ ]:
inception_predict(data.camera())